Metode moora adalah metode yang mengoptimisasi banyak objective yang berbasis analisis ratio. Cara kerja metode ini adalah memberikan bobot pada setiap kriteria yang ditentukan.

In [21]:
import pandas as pd
import numpy as np

In [22]:
path = "dataset/dataset_clean.csv"
df = pd.read_csv(path)
# df = df.head(4)
print(df)

                         Corporation  ROE - Return On Equity  \
0          American States Water Co.                  8.1724   
1     Automatic Data Processing Inc.                 22.0354   
2                         Avnet Inc.                 13.6376   
3       California Water Service Co.                  9.6412   
4               Cardinal Health Inc.                 11.1256   
...                              ...                     ...   
1372                   Welbilt, Inc.                 12.9974   
1373         Westlake Chemical Corp.                 18.6697   
1374                     Xerox Corp.                  9.7833   
1375            YPF Sociedad Anonima                  3.6742   
1376       iHeartCommunications Inc.                 11.8335   

      Net Profit Margin  Operating Cash Flow Per Share  Debt/Equity Ratio  
0                8.3224                         1.9957             0.8847  
1               13.5690                         0.2501             0.0073  
2  

normalisasi

In [23]:
def normalize_moora(matrix):
    return matrix / np.sqrt(np.sum(matrix**2, axis=0))

Fungsi perhitungan dengan moora

In [24]:
def moora_method(matrix, weights, benefit_criteria):
    # Normalisasi matrix
    normalized = normalize_moora(matrix)
    
    # Mengalikan dengan bobot
    weighted = normalized * weights
    
    # Menghitung skor untuk kriteria benefit dan cost
    benefit_sum = np.sum(weighted[:, benefit_criteria], axis=1)
    cost_sum = np.sum(weighted[:, [i for i in range(weighted.shape[1]) if i not in benefit_criteria]], axis=1)
    
    # Menghitung skor akhir (benefit - cost)
    moora_score = benefit_sum - cost_sum
    
    return moora_score

Menentukan kriteria, bobot dan benefit

In [25]:
criteria = ['ROE - Return On Equity', 'Net Profit Margin', 'Operating Cash Flow Per Share', 'Debt/Equity Ratio']
weights = np.array([0.30, 0.25, 0.20, 0.25])
benefit_criteria = [0, 1, 2]  # Indeks kolom untuk kriteria benefit (0-based)

membuat matrix

In [26]:
matrix = df[criteria].values

Perhitungan skor dengan mooora

In [27]:
scores = moora_method(matrix, weights, benefit_criteria)

# Menambahkan skor ke dataframe
df['Score'] = scores

# Mengurutkan berdasarkan skor
df_sorted = df.sort_values('Score', ascending=False).reset_index(drop=True)

print("Hasil Perangkingan:")
print(df_sorted[['Corporation', 'Score'] + criteria])

# Rekomendasi
print("\nRekomendasi:")
print(f"Berdasarkan analisis MOORA, perusahaan {df_sorted.iloc[0]['Corporation']} memiliki skor tertinggi dan merupakan pilihan terbaik untuk investasi.")

# Analisis tambahan
print("\nAnalisis Perbandingan:")
best_score = df_sorted.iloc[0]['Score']
worst_score = df_sorted.iloc[-1]['Score']
score_range = best_score - worst_score

for i, row in df_sorted.iterrows():
    if i == 0:
        continue
    relative_performance = (row['Score'] - worst_score) / score_range
    print(f"Perusahaan {row['Corporation']} memiliki {relative_performance:.2%} dari rentang kinerja relatif terhadap perusahaan terbaik dan terburuk.")

Hasil Perangkingan:
                                        Corporation     Score  \
0                            Nabors Industries Inc.  0.154287   
1                       LOCKHEED MARTIN CORPORATION  0.122557   
2            Entravision Communications Corporation  0.100747   
3                               LIBERTY MEDIA CORP.  0.065281   
4                                   CDW Corporation  0.054170   
...                                             ...       ...   
1372                            NEKTAR THERAPEUTICS -0.068184   
1373                              Pitney Bowes Inc. -0.072686   
1374                 Pacific Life Insurance Company -0.087652   
1375  Global One Real Estate Investment Corporation -0.088105   
1376                      Eldorado Gold Corporation -0.092874   

      ROE - Return On Equity  Net Profit Margin  \
0                     2.5612            -2.4218   
1                  7038.4610             5.8179   
2                  1461.7430            -4.218